In [7]:
import numpy as np
import pandas as pd
import copy, pickle
from datetime import datetime


In [3]:
#charginig function
def charge(x,a,b):
    return((a+b*x)*(x>0))

In [93]:
# #schedule: each pi_i has an ordered set of jobs(j)
# piis = [[]]*m
# #charging amount each x_i has an ordered set of charging times
# xis = [[]]*m
def random_schedule(P):
#     #1.partition jobs into m machines, 2.sequence them, 3.assign charging times
#     piis = [[]]*m
#     #charging amount each x_i has an ordered set of charging times
#     xis = [[]]*m
    pjs = P['p']
    Bis = P['B']
    Bios = P['Bo']
    n = len(pjs)
    m = len(Bis)
    jobs = np.random.permutation(np.arange(n))
    partitions = [0]+sorted(np.random.choice(n, m-1))+[n]
    piis = [jobs[partitions[i]:partitions[i+1]] for i in range(len(partitions)-1)]
    
    raw_xis = [np.random.normal(10,1,len(pi)) for pi in piis]
    xis = [raw_xis[i]*max(pjs[piis[i]].sum()-Bios[i],0)/raw_xis[i].sum() for i in range(len(raw_xis))]
    return(piis, xis)

def spt_schedule(P):
    pass
    
def edd_schedule(P):    
    pass

def tF1(job, pk, wk, S, B0, b):
    return([S[0], S[1]+pk, S[2]+wk, S[3]+wk*(B0+(1+b)*(S[1]+pk)), S[4],S[5]+[job]])

def tF2(job, pk, wk, S):
    return([S[0]+pk, S[1], S[2], S[3]+wk*(S[0]+pk), S[4]+[job], S[5]])

def alg1(P, straddling):
    pjs = P['p']
    wjs = P['w']
    Bis = P['B']
    Bios = P['Bo']
    a = P['a']
    b = P['b']
    n = len(pjs)
    m = len(Bis)
    
    sorted_jobs = np.argsort(pjs/wjs)
    S_1 = [[0,0,0,0,[],[]]]
    for k in range(n):
        job = sorted_jobs[k]
        if job==straddling:
            continue
        pk = pjs[job]
        wk = wjs[job]
        S_0 = []
        for S in S_1:
            S_0 = S_0 + [tF1(job, pk, wk, S, Bios[0], b)]
            if S[0]+pk <= Bios[0]:
                S_0 = S_0 + [tF2(job, pk, wk, S)]
        S_1 = copy.deepcopy(S_0)
    
    minG = float('inf') 
    S_before = []
    S_after = []
    for S in S_0:
        if S[0] <= Bios[0] and S[0]+pjs[straddling] > Bios[0]:
            G = S[3]+wjs[straddling]*Bios[0]+(S[2]+wjs[straddling])*(1+b)*(S[0]+pjs[straddling]-Bios[0])
            if G<minG:
                minG = G
                S_before = S[4]
                S_after = S[5]
    return(minG, S_before+[straddling]+S_after)
        

def cost_func(P, piis, xis, mode, w=None): #mode 0: completion, mode 1:binary-tardiness
    pjs = P['p']
    wjs = P['w']
    djs = P['d']
    if w is not None:
        wjs = w
    a = P['a']
    b = P['b']
    cost = 0
    for i in range(len(piis)):
        compl = pjs[piis[i]].cumsum()+charge(xis[i],a,b).cumsum() 
#         print(compl,djs[piis[i]])
#         print((compl>djs[piis[i]]).sum())
        if mode==0:
            cost += (wjs[piis[i]]*compl).sum()
        elif mode==1:
            cost += (wjs[piis[i]]*(compl>djs[piis[i]])).sum()
    return(cost)
        
def get_x(pi, pjs, Bo):
    strd = np.where(pjs[pi].cumsum()>Bo)[0][0]
#     print(pjs[pi],strd)
#     print(pi)
    xi = np.concatenate((np.maximum(pjs[pi].cumsum()-Bo,0)[:(strd+1)],pjs[pi][(strd+1):]))
    return(xi)
    
def alg1_prime(P, straddling):
    pjs = P['p']
    wjs = P['w']
    Bis = P['B']
    Bios = P['Bo']
    a = P['a']
    b = P['b']
    n = len(pjs)
    m = len(Bis)
    
    sorted_jobs = np.argsort(pjs/wjs).tolist()
    sorted_jobs.remove(straddling)
    fringe = list(zip(*[[[x] for x in sorted_jobs],pjs[sorted_jobs]]))
    feasibles = []
    min_cost = float('inf')
    schedule = ([],[])
    while len(fringe)>0:
        node = fringe.pop()
        #evaluate
        if node[1]+pjs[straddling] > Bios[0]:
            feasibles.append(node)
            pi = node[0]+[straddling]+[x for x in sorted_jobs if x not in node[0]]
#             print(np.zeros(n)[pjs[pi].cumsum()<=Bios[0]])
#             print(pjs[pi][pjs[pi].cumsum()>Bios[0]])
#             print(pjs[pi].cumsum()<=Bios[0])
#             xi = np.concatenate(np.zeros(n)[pjs[pi].cumsum()<=Bios[0]], pjs[pi][pjs[pi].cumsum()>Bios[0]])
#             xi = np.concatenate((np.maximum(pjs[pi].cumsum()-Bios[0],0)[:(pi.index(straddling)+1)],pjs[pi][(pi.index(straddling)+1):]))
            
            xi = get_x(pi, pjs, Bios[0])
            cost = cost_func(P, [pi], [xi], 0)
#             print(pi,cost)
            if cost < min_cost:
                min_cost = cost
                schedule = (pi,xi)
        #expand
        last_job = node[0][-1]
        next_candis = sorted_jobs[(sorted_jobs.index(last_job)+1):]
        for job in next_candis:
            child = (node[0]+[job], node[1]+pjs[job])
            if child[1] <= Bios[0]:#-pjs[straddling]:
                fringe.append(child)

                
    return(min_cost, schedule)
        

def alg1_wspt(P, straddling):
    pjs = P['p']
    wjs = P['w']
    Bis = P['B']
    Bios = P['Bo']
    a = P['a']
    b = P['b']
    n = len(pjs)
    m = len(Bis)
    
    S_after = np.argsort(pjs/wjs).tolist()
    S_after.remove(straddling)
    
    sk = 0
    S_before = []
    pis = []
    while len(S_after)>0:
#     for job in sorted_jobs:
        if sk > Bios[0]:
            break
        if sk+pjs[straddling]>Bios[0]:
            pis.append(S_before+[straddling]+S_after)
        job = S_after.pop(0)
        sk += pjs[job]
        S_before.append(job)
        
    min_cost = float('inf')
    schedule = ([],[])
    
    for pi in pis:
        xi = get_x(pi, pjs, Bios[0])
        cost = cost_func(P, [pi], [xi], 0)
#         print(pi, cost)
        if cost < min_cost:
            min_cost = cost
            schedule = (pi,xi)
    
    return(min_cost, schedule)
        
def knapSack(W, wt, val, n):
    K = [[0 for x in range(W + 1)] for x in range(n + 1)]
 
    # Build table K[][] in bottom up manner
    insack = [[[] for x in range(W+1)] for x in range(n+1)]
    for i in range(n + 1):
        for w in range(W + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
             
            elif wt[i-1] <= w:
                val_i_1 = val[i-1]+ K[i-1][w-wt[i-1]]
                if val_i_1>=K[i-1][w]:
                    K[i][w] = val_i_1
                    insack[i][w] = insack[i-1][w-wt[i-1]]+[i-1]
                else:
                    K[i][w] = K[i-1][w]
                    insack[i][w] = insack[i-1][w]
#                 K[i][w] = max(val[i-1]
#                           + K[i-1][w-wt[i-1]], 
#                               K[i-1][w])
            else:
                K[i][w] = K[i-1][w]
                insack[i][w] = insack[i-1][w]
 
    return (insack[n][W], K[n][W])
 
 
# Driver code
val = [60, 50, 120]
wt = [10, 20, 30]
W = 50
n = len(val)
print(knapSack(W, wt, val, n))
    
def alg1_knapsack(P, straddling):
    pjs = P['p']
    wjs = P['w']
    Bis = P['B']
    Bios = P['Bo']
    a = P['a']
    b = P['b']
    n = len(pjs)
    m = len(Bis)
    
    all_jobs = np.argsort(pjs/wjs).tolist()
    all_jobs.remove(straddling)
    insack, _ = knapSack(Bios[0], pjs, wjs, n)
    S_before = [insack[x] for  x in np.argsort(pjs[insack]/wjs[insack]).tolist()]
    S_after = [x for x in all_jobs if x not in insack]
    
    pi = S_before+[straddling]+S_after
    xi = get_x(pi, pjs, Bios[0])
    cost = cost_func(P, [pi], [xi], 0)
    return(cost, (pi,xi))
    
    

([0, 2], 180)


In [5]:
def moore(P):
    pjs = P['p']
    wjs = P['w']
    djs = P['d']
    Bis = P['B']
    Bios = P['Bo']
    a = P['a']
    b = P['b']
    n = len(pjs)
    m = len(Bis)
    
    sorted_jobs = np.argsort(djs).tolist()
    pi = copy.copy(sorted_jobs)
    xi = get_x(pi, pjs, Bios[0])
    T = []
    
    compl = pjs[pi].cumsum()+charge(xi,a,b).cumsum()
    tardiness = np.maximum(compl-djs[pi],0)
    
    while (tardiness>0).sum()>0:
        print(pi, T)
        job_a = np.where(tardiness>0)[0][0]
        job_b = pi[np.argmax(pjs[pi][:(job_a+1)])]
        pi.remove(job_b)
        T.append(job_b)
        
        xi = get_x(pi, pjs, Bios[0])
        compl = pjs[pi].cumsum()+charge(xi,a,b).cumsum()
        tardiness = np.maximum(compl-djs[pi],0)
#         print(pi, T)
        

    pi = pi + T
    xi = get_x(pi, pjs, Bios[0])
    compl = (pjs[pi].cumsum()+charge(xi,a,b).cumsum())
#     print(compl)
#     print((compl>djs[pi]).sum())
    return(pi, xi)

In [8]:
# print(pjs,djs)
import math
math.factorial(10)
# import pickle
# with open('CounterExampleOfWSPT.pickle', 'wb') as f:
#     pickle.dump(prob, f)
with open('CounterExampleOfWSPT.pickle', 'rb') as f:
    prob = pickle.load(f)

In [94]:
#Jobs: j from 0 to n, indexed by number
def makeprob(n, m):
#     n = 20
    pjs = np.random.normal(10,2,n).astype(int)
    djs = np.random.normal(50,10,n).astype(int)
    wjs = (np.random.rand(n)*10).astype(int)+1

    #machines
#     m = 1
    Bis = n/m*np.random.normal(10,1,m).astype(int)
    Bios = (Bis/3).astype(int)

    a = 0
    b = 1
    prob = {'p':pjs, 'd':djs, 'w':wjs, 'B':Bis, 'Bo':Bios, 'a':a, 'b':b}
#     print('wspt:',np.argsort(pjs/wjs).tolist())
#     print('edd:',np.argsort(djs).tolist())
#     print('p:', pjs)
    return(prob)
n = 10
m = 1
prob = makeprob(n,m)
print(prob)
alg1_knapsack(prob, 1)
# (prob['Bo'])
# print(np.random.normal(10,1,2))
# prob['Bo'][0]

{'p': array([ 7,  9,  8,  9,  7, 10,  9, 12, 12, 11]), 'd': array([49, 45, 37, 63, 62, 42, 46, 64, 38, 49]), 'w': array([7, 2, 7, 5, 9, 8, 1, 2, 1, 2]), 'B': array([90.]), 'Bo': array([30]), 'a': 0, 'b': 1}


(1866,
 ([4, 2, 5, 1, 0, 3, 9, 7, 6, 8],
  array([ 0,  0,  0,  4,  7,  9, 11, 12,  9, 12])))

In [17]:
pi, xi = moore(prob)
print(pi,xi)
cost_func(prob, [pi], [xi], 1, np.ones(len(prob['p'])))
prob['Bo']


[4, 8, 6, 1, 3, 9, 2, 7, 5, 0] []
[4, 8, 6, 3, 9, 2, 7, 5, 0] [1]
[4, 8, 3, 9, 2, 7, 5, 0] [1, 6]
[8, 3, 9, 2, 7, 5, 0] [1, 6, 4]
[3, 9, 2, 7, 5, 0] [1, 6, 4, 8]
[3, 9, 2, 7, 5, 1, 6, 4, 8, 0] [ 0.          0.          0.          8.25316211  8.44884115 10.37047726
 10.35349699 10.02919775  9.85851905 10.35005803]


array([27.81396268])

In [96]:
piis, xis = random_schedule(prob)
t = datetime.now()
print(alg1(prob, 2))
print(datetime.now()-t)

(322.5464845227233, [5, 1, 6, 2, 3, 4, 8, 0, 7, 9])
0:00:00.027268


In [98]:
t = datetime.now()
print(alg1_prime(prob,2))
print(datetime.now()-t)

for k in range(100):
    print(k)
    prob = makeprob(n,m)

    min_costp = float('inf')
    opt_sp = []
    min_costw = float('inf')
    opt_sw = []
    for j in range(len(prob['p'])):
        costp, schedulep = alg1_prime(prob,j)
        if costp < min_costp:
            min_costp = costp
            opt_sp = schedulep
            
        costw, schedulew = alg1_knapsack(prob,j)
        if costw < min_costw:
            min_costw = costw
            opt_sw = schedulew            
#     print(min_cost,opt_s)
#     wspt = np.argsort(prob['p']/prob['w']).tolist()
    if opt_sp[0] != opt_sw[0]:
        print(prob)
        print(opt_sp, opt_sw, min_costp-min_costw)

        
schedule = random_schedule(prob)        
pi = schedule[0][0]
print(min_costp, min_costw)
cost_func(prob, [pi], [get_x(schedule[0][0], prob['p'], prob['Bo'])], 0, w=None)
# prob['Bo']
# pi,xi = alg1_prime(prob,1)[0]

(2423, ([4, 1, 9, 7, 2, 8, 3, 6, 5, 0], array([ 0,  0,  0,  0,  7, 10,  8, 12, 10, 10])))
0:00:00.017478
0
{'p': array([12,  8,  9,  9,  8, 11,  7,  9,  9, 10]), 'd': array([49, 60, 43, 59, 65, 41, 57, 63, 44, 65]), 'w': array([3, 5, 3, 4, 5, 6, 3, 1, 4, 9]), 'B': array([100.]), 'Bo': array([33]), 'a': 0, 'b': 1}
([9, 1, 4, 5, 3, 8, 6, 2, 0, 7], array([ 0,  0,  0,  4,  9,  9,  7,  9, 12,  9])) ([9, 1, 4, 6, 5, 3, 8, 2, 0, 7], array([ 0,  0,  0,  0, 11,  9,  9,  9, 12,  9])) -22
1
2
{'p': array([ 8, 13,  8, 11, 12,  9, 11, 11, 11,  9]), 'd': array([55, 54, 46, 50, 55, 53, 54, 45, 64, 40]), 'w': array([ 7,  3,  2,  7, 10,  2,  1,  6,  9,  7]), 'B': array([100.]), 'Bo': array([33]), 'a': 0, 'b': 1}
([0, 4, 8, 9, 3, 7, 2, 1, 5, 6], array([ 0,  0,  0,  7, 11, 11,  8, 13,  9, 11])) ([4, 8, 9, 0, 3, 7, 2, 1, 5, 6], array([ 0,  0,  0,  7, 11, 11,  8, 13,  9, 11])) -16
3
4
{'p': array([10,  6,  9,  8, 11,  6,  9, 12,  7, 11]), 'd': array([58, 39, 62, 45, 51, 38, 65, 45, 50, 46]), 'w': array([ 1

3558

In [25]:
a=[1,2,3]
b=[1,2,3]
a != b

False

In [368]:
t = datetime.now()
print(alg1_wspt(prob,1))
print(datetime.now()-t)

[3, 16, 0, 8, 1, 18, 9, 19, 2, 6, 15, 13, 14, 5, 7, 17, 12, 4, 10, 11] 992.0105130046152
(992.0105130046152, ([3, 16, 0, 8, 1, 18, 9, 19, 2, 6, 15, 13, 14, 5, 7, 17, 12, 4, 10, 11], array([ 0.        ,  0.        ,  0.        ,  0.        ,  4.3635351 ,
       10.01198102,  9.99029899,  9.84451154,  9.63228741,  9.14395625,
       11.6155721 ,  9.84701961, 10.05642563, 10.70791356,  9.45587469,
       10.74719516,  9.81307453, 10.44777235, 10.12983092,  7.85539266])))
0:00:00.002880


In [231]:
pjs[[8,0,1]].cumsum()

array([ 9.58698012, 20.05094611, 30.80500601])

In [222]:
strad = 1
pi = alg1(prob,strad)[1]
xi = np.concatenate((np.maximum(pjs[pi].cumsum()-Bios[0],0)[:(pi.index(strad)+1)],pjs[pi][(pi.index(strad)+1):]))

In [223]:

cost_func(prob,[pi],[xi],0)

323.0870151381639

In [230]:
Bios[0]-pjs[1]

19.146449970932434

In [206]:
pjs[pi].cumsum()+charge(xi,a,b).cumsum() 

array([  9.58698012,  20.05094611,  29.5821405 ,  50.77189093,
        71.41797836,  90.9760396 , 112.54861405, 132.70829209,
       154.50594569, 176.55356555])

In [203]:
(wjs[pi]*pjs[pi].cumsum()).sum()

229.06344579020313

In [122]:
x=list(zip(*[[[x] for x in sorted_jobs],pjs[sorted_jobs]]))

In [270]:
np.argsort(pjs/wjs).tolist()

[7, 11, 2, 3, 9, 16, 0, 4, 1, 19, 15, 18, 12, 10, 17, 13, 14, 6, 5, 8]

In [355]:
sorted_jobs = np.argsort(pjs/wjs).tolist()
# sorted_jobs.remove(straddling)
x=sorted_jobs
# x.remove(1)
x

[16, 9, 15, 5, 6, 18, 2, 1, 8, 17, 7, 12, 13, 0, 11, 14, 4, 19, 3, 10]

In [323]:
pjs[sorted_jobs]

array([ 6.94338995, 10.33064521,  8.71784087,  9.17875766, 10.96900673,
        9.34536401,  8.95928837,  9.54723498, 11.93809886,  9.19142603,
        9.75928587, 10.83273451,  9.93331367,  9.84329008, 10.56594293,
        9.07181298,  9.47634918, 10.32656257,  9.58805126, 12.38021537])

In [328]:
Bios[0]

9.559670397404512

In [377]:
x=np.array([1,2,3])
np.where(x>1)[0][0]
np.argmax(x)

2

In [378]:
x[:2]

array([1, 2])

In [381]:
np.zeros(4)

array([0., 0., 0., 0.])

In [382]:
x

array([1, 2, 3])

In [393]:
print(x>2)

[False False  True]


In [399]:
np.zeros(3)[pjs[[1,2,3]].cumsum()>10]

array([0., 0.])